# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [53]:
! git clone https://github.com/badreouh/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

fatal: destination path 'exam_2025' already exists and is not an empty directory.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [54]:
mySeed = 333

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [55]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

In [59]:

#On peut utiliser la regression linéaire

from sklearn.linear_model import LinearRegression
# Extract inputs and targets from the training set
X_train = train_set['inputs']
y_train = train_set['targets']

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# coefficients
theta0 = model.intercept_
theta1, theta2, theta3 = model.coef_

# the results
print(f"Estimated coefficients:")
print(f"θ0 (Intercept): {theta0}")
print(f"θ1 (x): {theta1}")
print(f"θ2 (y): {theta2}")
print(f"θ3 (z): {theta3}")


Estimated coefficients:
θ0 (Intercept): 16.558024510983287
θ1 (x): 3.2729838244521834
θ2 (y): 3.294835451763797
θ3 (z): 6.687036331785982


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

In [60]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 1)  # Linear layer for 3 inputs (x, y, z) and 1 output (t)

    def forward(self, x):
        return self.fc(x)  # Return the linear combination

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [58]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    epoch_loss = 0.0  # Track total loss for this epoch
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()  # Reset gradients to zero

        # Forward pass: compute predictions
        outputs = mySimpleNet(batch_inputs)

        # Compute the loss
        loss = criterion(outputs.squeeze(), batch_targets)

        # Backward pass: compute gradients
        loss.backward()

        # Update parameters
        optimizer.step()

        # Accumulate loss for reporting
        epoch_loss += loss.item()

    # Print the epoch loss
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/500], Loss: 3500.9886
Epoch [2/500], Loss: 2085.3168
Epoch [3/500], Loss: 1254.0649
Epoch [4/500], Loss: 764.5841
Epoch [5/500], Loss: 476.0548
Epoch [6/500], Loss: 305.6546
Epoch [7/500], Loss: 204.4943
Epoch [8/500], Loss: 144.1420
Epoch [9/500], Loss: 107.9004
Epoch [10/500], Loss: 85.9064
Epoch [11/500], Loss: 72.3529
Epoch [12/500], Loss: 63.8975
Epoch [13/500], Loss: 58.4909
Epoch [14/500], Loss: 54.9151
Epoch [15/500], Loss: 52.4693
Epoch [16/500], Loss: 50.7157
Epoch [17/500], Loss: 49.4094
Epoch [18/500], Loss: 48.3847
Epoch [19/500], Loss: 47.5643
Epoch [20/500], Loss: 46.8610
Epoch [21/500], Loss: 46.2617
Epoch [22/500], Loss: 45.7477
Epoch [23/500], Loss: 45.2759
Epoch [24/500], Loss: 44.8481
Epoch [25/500], Loss: 44.4586
Epoch [26/500], Loss: 44.1075
Epoch [27/500], Loss: 43.7912
Epoch [28/500], Loss: 43.5048
Epoch [29/500], Loss: 43.2220
Epoch [30/500], Loss: 42.9886
Epoch [31/500], Loss: 42.7563
Epoch [32/500], Loss: 42.5404
Epoch [33/500], Loss: 42.3366
Epoch [

**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [61]:
# Extract the trained parameters (weights and bias) from the model
theta0 = mySimpleNet.fc.bias.item()  # Intercept (θ0)
theta1, theta2, theta3 = mySimpleNet.fc.weight[0].detach().numpy()  # Coefficients (θ1, θ2, θ3)

# Print the estimated parameters
print("Estimated coefficients:")
print(f"θ0 : {theta0}")
print(f"θ1 (x): {theta1}")
print(f"θ2 (y): {theta2}")
print(f"θ3 (z): {theta3}")


Estimated coefficients:
θ0 (Intercept): 16.557300567626953
θ1 (x): 3.272944450378418
θ2 (y): 3.2953481674194336
θ3 (z): 6.686279296875


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [44]:
# Extract inputs and targets from the test set
X_test = test_set['inputs']
y_test = test_set['targets']

# Linear Regression Predictions
y_pred_lr = model.predict(X_test)
mse_lr = ((y_pred_lr - y_test) ** 2).mean()

# Neural Network Predictions
mySimpleNet.eval()  # Set the neural network to evaluation mode
with torch.no_grad():  # Disable gradient computation
    y_pred_nn = mySimpleNet(torch.tensor(X_test, dtype=torch.float32)).squeeze().numpy()
mse_nn = ((y_pred_nn - y_test) ** 2).mean()

# Compare the Mean Squared Errors
print(f"Linear Regression Test MSE: {mse_lr:.4f}")
print(f"Neural Network Test MSE: {mse_nn:.4f}")


Linear Regression Test MSE: 3.9932
Neural Network Test MSE: 3.9937


**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [46]:

# Nb de paramètres dans self.Down1: (calcul "à la main")

params_conv1 = (128 * 64 * 3) + 128  # First Conv1d layer in Double_conv_causal
params_conv2 = (128 * 128 * 3) + 128  # Second Conv1d layer in Double_conv_causal
params_bn1 = (128 * 2)  # Batch Norm1d (128 weights + 128 biases)
params_bn2 = (128 * 2)  # Batch Norm1d (128 weights + 128 biases)
params_down1_total = params_conv1 + params_conv2 + params_bn1 + params_bn2
print(f"Nb de paramètres dans self.Down1: {params_down1_total}")

# Nb de paramètres au total:
model = causalFCN()
total_parameters = sum(p.numel() for p in model.parameters())
print(f"Nb de paramètres au total: {total_parameters}")

Nb de paramètres dans self.Down1: 74496
Nb de paramètres au total: 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

#Réponse :
La taille du vecteur d’entrée est réduite par le biais de max-pooling dans des couches telles que self.down1, qui sous-échantillonnent la résolution temporelle, et grâce à l’augmentation de la profondeur des canaux dans les couches de convolution. Elle est ensuite restaurée dans la seconde partie du réseau à l’aide de convolutions transposées dans des couches comme self.up2, qui effectuent un suréchantillonnage de la résolution temporelle, ainsi que par des connexions de saut (skip connections) qui combinent des caractéristiques haute résolution issues des couches précédentes avec la sortie suréchantillonnée.

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Le champ récepteur est augmenté grâce à la dilatation, à la taille du noyau dans les couches de convolution et au max-pooling dans la voie de sous-échantillonnage. Pour self.inc, qui utilise deux couches de convolution avec une taille de noyau de 3 et une dilatation de 1, le champ récepteur augmente comme suit : la première convolution a un champ récepteur de 3, et la seconde convolution l’étend à
3+(3−1)=5. Par conséquent, le champ récepteur de self.inc est 5.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [49]:
# Create 2 inputs
input1 = torch.zeros(1, 1, 10000)
input2 = input1.clone()
input2[0, 0, 4995] = 1  # Modify a point near index 5000

# Pass inputs through the network
output1 = model(input1)
output2 = model(input2)

# Find the indices where the output at 5000 differs
diff = torch.abs(output1 - output2)
field_receptive = (diff[0, 0, :] > 1e-5).nonzero().max() - (diff[0, 0, :] > 1e-5).nonzero().min() + 1

print(f"Receptive field size: {field_receptive.item()}")


Receptive field size: 10000


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



In [51]:
# Create two inputs
input1 = torch.zeros(1, 1, 10000)
input2 = input1.clone()
input2[0, 0, 5006] = 1  # Modify a point after index 5000

# Pass inputs through the network
output1 = model(input1)
output2 = model(input2)

# Check if there are any changes
y5000_diff = torch.abs(output1[0, 0, 5000] - output2[0, 0, 5000]).item()
print(f"y[5000] difference: {y5000_diff}")


y[5000] difference: 0.01576147973537445



#RÉPONSE :

\( y_{5000} \) ne dépend pas de \( x_t \) pour \( t > 5000 \), et \( y_{5000\_diff} \) est quasiment nul, ce qui confirme la causalité.

La propriété de « causalité » est garantie par le padding manuel dans la méthode forward de **Double_conv_causal** :
```python
x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))


\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

#REPONSE :
Les échantillons positifs correspondent aux éléments jugés pertinents par rapport à une requête, comme un résultat de recherche particulièrement adapté à la demande de l’utilisateur. De leur côté, les échantillons négatifs représentent des éléments qui ne répondent pas ou répondent peu à la requête, tels qu’un résultat de recherche inapproprié ou sans rapport.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  



#REPONSE :  
Dans \\(\\mathcal{L}_{RankNet}\\), chaque document \\(i\\) se voit attribuer un score \\(z_i\\) par le modèle, représentant sa pertinence estimée pour la requête. En pratique, \\(z_i\\) est produit par une fonction de scoring (souvent un réseau de neurones). Ces notes de pertinence servent ensuite à comparer deux documents à la fois, via la différence \\(z_i - z_j\\), afin de déterminer leur ordre de pertinence. C’est cette comparaison par paires qui sous-tend la fonction de perte de RankNet.
"""

print(texte_reponse)


**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

 #REPONSE:

 L'expression pour la fonction de perte $\mathcal{L}_{RankNet}$ minimise la perte pair-à-pair en ajustant les scores prédits $z_i$ et $z_j$
 de manière à ce que $\sigma(z_i - z_j)$ approche 1 lorsque $y_{ij} = 1$, ce qui signifie que le modèle prédit $z_i > z_j$
 pour une paire positive-négative. Inversement, $\sigma(z_i - z_j)$ se rapproche de 0 lorsque $y_{ij} = 0$, indiquant que $z_i < z_j$.
 Ce processus d'optimisation garantit qu'après l'apprentissage, les scores prédits sont correctement alignés avec les relations attendues
 entre les paires d'exemples.


**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?

#REPONSE :
Dans une approche d’apprentissage profond, les réseaux de neurones dédiés à ce type de tâche sont appelés réseaux de classement (ranking networks), comme RankNet, et sont conçus pour comparer la pertinence de paires ou d’ensembles de documents. L’entraînement de ces réseaux s’effectue selon une stratégie de pairwise learning (apprentissage par paires), où l’objectif est de minimiser une fonction de perte basée sur l’écart de scores entre deux documents, de sorte que les exemples positifs obtiennent des scores plus élevés que les exemples négatifs.